<a href="https://colab.research.google.com/github/nairsatish/4540/blob/master/Copy_of_demistifying.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

From article: https://www.analyticsvidhya.com/blog/2020/02/mathematics-behind-convolutional-neural-network/

In [44]:
#importing required libraries
import numpy as np
import pandas as pd
import tqdm as tqdm
from keras.datasets import mnist

#loading dataset
(x_train, y_train), (x_test,y_test) = mnist.load_data()

#selecting a subset of data (200 images)
x_train = x_train[:200]
y=y_train[:200]

X = x_train.T
X = X/255

y.resize((200,1))
y = y.T
#checking value
pd.Series(y[0]).value_counts()


1    26
9    23
0    21
4    21
3    21
7    21
2    20
6    19
8    15
5    13
dtype: int64

In [45]:
#converting into binary classification
for i in range(y.shape[1]):
    if y[0][i] >4:
        y[0][i] = 1
    else:
        y[0][i] = 0

#checking value counts
pd.Series(y[0]).value_counts()

0    109
1     91
dtype: int64

In [46]:
# initializing filter
f=np.random.uniform(size=(3,5,5))
f = f.T

print('Filter 1', '\n', f[:,:,0], '\n')
print('Filter 2', '\n', f[:,:,1], '\n')
print('Filter 3', '\n', f[:,:,2], '\n')

Filter 1 
 [[0.12559085 0.38527453 0.90251958 0.40887072 0.38244406]
 [0.1112728  0.23616078 0.69531019 0.28636856 0.75975435]
 [0.08699854 0.76027244 0.99761362 0.78619734 0.0663354 ]
 [0.3393071  0.61420234 0.41146288 0.8538644  0.16719704]
 [0.58416605 0.307486   0.88213476 0.0418527  0.19331886]] 

Filter 2 
 [[0.38004315 0.06401921 0.76834575 0.72079877 0.18515008]
 [0.89161016 0.8718618  0.17198951 0.3483645  0.95221048]
 [0.79553685 0.35399886 0.12797227 0.61603282 0.51773285]
 [0.05169041 0.51011704 0.60314608 0.99198456 0.55839141]
 [0.69210159 0.00555821 0.28980714 0.21289428 0.20960447]] 

Filter 3 
 [[0.4066338  0.02351666 0.89389397 0.03732026 0.51821497]
 [0.63789561 0.32017307 0.78482944 0.64072335 0.69107697]
 [0.9021444  0.16917474 0.14989648 0.35686705 0.66417767]
 [0.81416498 0.25144645 0.60491165 0.88224277 0.10961808]
 [0.68190011 0.53350247 0.09033398 0.62507958 0.24431668]] 



In [47]:
X.shape, y.shape, f.shape

((28, 28, 200), (1, 200), (5, 5, 3))

In [48]:
# Generating patches from images
new_image = []

# for number of images
for k in range(X.shape[2]):
    # sliding in horizontal direction
    for i in range(X.shape[0]-f.shape[0]+1):
        # sliding in vertical direction
        for j in range(X.shape[1]-f.shape[1]+1):
            new_image.append(X[:,:,k][i:i+f.shape[0],j:j+f.shape[1]])
            
# resizing the generated patches as per number of images
new_image = np.array(new_image)
new_image.resize((X.shape[2],int(new_image.shape[0]/X.shape[2]),new_image.shape[1],new_image.shape[2]))
new_image.shape

(200, 576, 5, 5)

In [49]:
# number of features in data set
s_row = X.shape[0] - f.shape[0] + 1
s_col = X.shape[1] - f.shape[1] + 1
num_filter = f.shape[2]

inputlayer_neurons = (s_row)*(s_col)*(num_filter)
output_neurons = 1 

# initializing weight
wo=np.random.uniform(size=(inputlayer_neurons,output_neurons))

In [50]:
# defining the Sigmoid Function
def sigmoid (x):
    return 1/(1 + np.exp(-x))

# derivative of Sigmoid Function
def derivatives_sigmoid(x):
    return x * (1 - x)

In [51]:
# generating output of convolution layer
filter_output = []
# for each image
for i in range(len(new_image)):
    # apply each filter
    for k in range(f.shape[2]):
        # do element wise multiplication
        for j in range(new_image.shape[1]):
            filter_output.append((new_image[i][j]*f[:,:,k]).sum()) 

filter_output = np.resize(np.array(filter_output), (len(new_image),f.shape[2],new_image.shape[1]))

# applying activation over convolution output
filter_output_sigmoid = sigmoid(filter_output)

filter_output.shape, filter_output_sigmoid.shape

((200, 3, 576), (200, 3, 576))

In [52]:
#for element in filter_output:
#   print (type(element))
print(filter_output.dtype)

float64


In [53]:
# generating input for fully connected layer
filter_output_sigmoid = filter_output_sigmoid.reshape((filter_output_sigmoid.shape[0],filter_output_sigmoid.shape[1]*filter_output_sigmoid.shape[2]))

filter_output_sigmoid = filter_output_sigmoid.T

# Linear trasnformation for fully Connected Layer
output_layer_input= np.dot(wo.T,filter_output_sigmoid)
output_layer_input = (output_layer_input - np.average(output_layer_input))/np.std(output_layer_input)

# activation function
output = sigmoid(output_layer_input)

In [61]:
filter_output_sigmoid.shape

(1728, 200)

In [54]:
#Error
error = np.square(y-output)/2

#Error w.r.t Output (Gradient)
error_wrt_output = -(y-output)

#Error w.r.t sigmoid transformation (output_layer_input)
output_wrt_output_layer_input=output*(1-output)

#Error w.r.t weight
output_wrt_w=filter_output_sigmoid

In [55]:
#delta change in w for fully connected layer
delta_error_fcp = np.dot(output_wrt_w,(error_wrt_output * output_wrt_output_layer_input).T)
lr = 0.01
wo = wo - lr*delta_error_fcp

In [56]:
#Error w.r.t sigmoid output
output_layer_input_wrt_filter_output_sigmoid = wo.T

#Error w.r.t sigmoid transformation
filter_output_sigmoid_wrt_filter_output = filter_output_sigmoid * (1-filter_output_sigmoid)

# cvalculating derivatives for backprop convolution
error_wrt_filter_output = np.dot(output_layer_input_wrt_filter_output_sigmoid.T,error_wrt_output*output_wrt_output_layer_input) * filter_output_sigmoid_wrt_filter_output
error_wrt_filter_output = np.average(error_wrt_filter_output, axis=1)
error_wrt_filter_output = np.resize(error_wrt_filter_output,(X.shape[0]-f.shape[0]+1,X.shape[1]-f.shape[1]+1, f.shape[2]))

In [57]:
filter_update = []
for i in range(f.shape[2]):
    for j in range(f.shape[0]):
        for k in range(f.shape[1]):            
            temp = 0
            spos_row = j
            spos_col = k
            epos_row = spos_row + s_row
            epos_col = spos_col + s_col
            for l in range(X.shape[2]):
                temp = temp + (X[spos_row:epos_row,spos_col:epos_col,l]*error_wrt_filter_output[:,:,i]).sum()
            filter_update.append(temp/X.shape[2])  

filter_update_array = np.array(filter_update)
filter_update_array = np.resize(filter_update_array,(f.shape[2],f.shape[0],f.shape[1]))

In [58]:
for i in range(f.shape[2]):
    f[:,:,i] = f[:,:,i] - lr*filter_update_array[i]

In [59]:
#EXAMPLE PYTHON SNIPPETS THAT MAY BE HELPFUL

In [60]:
lst = ['string', 1, 2, 'another string']
for element in lst:
   print (type(element))

<class 'str'>
<class 'int'>
<class 'int'>
<class 'str'>
